In [1]:
from drone_detector.utils import *
from drone_detector.imports import *
import os
from drone_detector.metrics import *
import warnings
warnings.filterwarnings("ignore")
sys.path.append('..')
from src.postproc_functions import *
from tqdm.auto import tqdm
tqdm.pandas()

As patch-level data and results are not really useful for our purposes, here we run the predictions for larger virtual plots. Each plot is tiled to 512x512px patches, possibly with 256px overlap and afterwards the predictions are collated and optionally cleaned so that the amount of overlapping predictions is lower.

# Hiidenportti test set

As Hiidenportti test set is so small, we can run predictions here if needed.

## No overlap, no post-processing

In [2]:
from drone_detector.engines.detectron2.predict import predict_instance_masks
raw_path = Path('../../data/raw/hiidenportti/virtual_plots/buffered_test/images')
test_rasters = [raw_path/f for f in os.listdir(raw_path) if f.endswith('tif')]

Template folder has the following structure:

```
template_folder
|-predicted_vectors
|-raster_tiles
|-vector_tiles
|-raw_preds
```

Where `raster_tiles` and `vector_tiles` are symbolic links pointing to corresponding data directories, and `predicted_vectors` and `raw_preds` are empty folders for predictions.

In [3]:
pred_outpath = Path('../results/hp_unprocessed/')
if not os.path.exists(pred_outpath):
    shutil.copytree('../results/template_folder/', pred_outpath, symlinks=True)

In [4]:
#| output: false

for t in test_rasters:
    outfile_name = pred_outpath/f'raw_preds/{str(t).split("/")[-1][:-4]}.geojson'
    predict_instance_masks(path_to_model_config='../models/hiidenportti/mask_rcnn_R_101_FPN_3x/config.yaml', 
                           path_to_image=str(t),
                           outfile=str(outfile_name),
                           processing_dir='temp',
                           tile_size=512,
                           tile_overlap=0,
                           smooth_preds=False,
                           use_tta=True,
                           coco_set='../../data/processed/hiidenportti/hiidenportti_valid.json',
                           postproc_results=False)

Reading and tiling ../../data/raw/hiidenportti/virtual_plots/buffered_test/images/104_49_Hiidenportti_Chunk5_orto.tif to 512x512 tiles with overlap of 0px


0it [00:00, ?it/s]

Loading model
Starting predictions


  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

...

Loading model
Starting predictions


  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

199 polygons before non-max suppression
199 polygons after non-max suppression
Removing intermediate files


In [5]:
raw_res_path = pred_outpath
truth_shps = sorted([raw_res_path/'vector_tiles'/f for f in os.listdir(raw_res_path/'vector_tiles')])
raw_shps = sorted([raw_res_path/'raw_preds'/f for f in os.listdir(raw_res_path/'raw_preds')])
rasters = sorted([raw_res_path/'raster_tiles'/f for f in os.listdir(raw_res_path/'raster_tiles')])

"Raw" predictions are modified as such:
1. Invalid polygons are fixed to be valid polygons. MultiPolygon masks are replaced with the largest single polygon of the multipoly.
2. Extent is clipped to be same as the corresponding ground truth data
3. Label numbering is adjusted
4. Polygons with area less than 16² pixels are discarded

In [6]:
for p, t in zip(raw_shps, truth_shps):
    temp_pred = gpd.read_file(p)
    temp_truth = gpd.read_file(t)
    temp_pred['geometry'] = temp_pred.apply(lambda row: fix_multipolys(row.geometry) 
                                            if row.geometry.type == 'MultiPolygon' 
                                            else shapely.geometry.Polygon(row.geometry.exterior), axis=1)
    temp_pred['label'] += 1
    temp_pred = gpd.clip(temp_pred, box(*temp_truth.total_bounds))
    temp_pred = temp_pred[temp_pred.geometry.area > 16*0.04**2]
    temp_pred.to_file(raw_res_path/'predicted_vectors'/p.name)

In [7]:
pred_shps = sorted([raw_res_path/'predicted_vectors'/f for f in os.listdir(raw_res_path/'predicted_vectors')])

Collate predictions and annotations so that IoU and such is easy to compute.

In [8]:
truths = None
preds = None

for p, t in zip(pred_shps, truth_shps):
    temp_pred = gpd.read_file(p)
    temp_truth = gpd.read_file(t)
    if truths is None:
        truths = temp_truth
        preds = temp_pred
    else:
        truths = pd.concat((truths, temp_truth))
        preds = pd.concat((preds, temp_pred))

Fix labeling.

In [9]:
preds['layer'] = preds.apply(lambda row: 'groundwood' if row.label == 2 else 'uprightwood', axis=1)

Check the number of predictions. The models have found almost 1500 more deadwood instances at this point.

In [10]:
preds.shape, truths.shape

((3179, 4), (1741, 6))

In [11]:
preds.label.value_counts()

2    2832
1     347
Name: label, dtype: int64

In [12]:
dis_truths = truths.dissolve(by='layer')
dis_preds = preds.dissolve(by='layer')

Check IoU-score.

In [13]:
poly_IoU(dis_truths, dis_preds)

layer
groundwood     0.473983
uprightwood    0.497028
dtype: float64

Run GisCOCOeval, which converts georeferenced vector files to COCO-annotations and runs the metrics.

In [14]:
deadwood_categories = [{'supercategory': 'deadwood', 'id':1, 'name':'uprightwood'},
                 
                       {'supercategory': 'deadwood', 'id':2, 'name':'groundwood'}]

raw_coco_eval = GisCOCOeval(raw_res_path, raw_res_path, 
                            None, None, deadwood_categories)

In [15]:
raw_coco_eval.prepare_data(gt_label_col='layer')

0it [00:00, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [16]:
raw_coco_eval.prepare_eval()

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.12s)
creating index...
index created!


As the virtual plots can contain more than 1000 annotations, set `maxDets` to larger values than default.

In [17]:
raw_coco_eval.coco_eval.params.maxDets = [1000, 10000]

In [18]:
raw_coco_eval.evaluate()


Evaluating for category uprightwood
Running per image evaluation...
Evaluate annotation type *segm*
DONE (t=0.73s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=10000 ] = 0.255
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=10000 ] = 0.528
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=10000 ] = 0.223
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=10000 ] = 0.110
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=10000 ] = 0.315
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=10000 ] = 1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.351
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.204
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.408
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 1.000

Evaluating for 

Compared to the patch-level results, the AP50 score is around 0.2 lower than patch-level results. However, edges are not handled in any way in this processing level.

Get the number of false positives (FP), true positives (TP) and false negatives (FN). Object detection has infinite number of true negatives so we are not interested in them.

In [20]:
fp_cols = [f'FP_{np.round(i, 2)}' for i in np.arange(0.5, 1.04, 0.05)]
tp_cols = [f'TP_{np.round(i, 2)}' for i in np.arange(0.5, 1.03, 0.05)]
tp_truths = truths.copy()
tp_truths.rename(columns={'groundwood':'label'}, inplace=True)
truth_sindex = tp_truths.sindex
fp_preds = preds.copy()
pred_sindex = fp_preds.sindex
tp_truths[tp_cols] = tp_truths.progress_apply(lambda row: is_true_positive(row, fp_preds, pred_sindex), 
                                              axis=1, result_type='expand')
fp_preds[fp_cols] = fp_preds.progress_apply(lambda row: is_false_positive(row, tp_truths, truth_sindex,
                                                                            fp_preds, pred_sindex),
                                            axis=1, result_type='expand')

  0%|          | 0/1741 [00:00<?, ?it/s]

  0%|          | 0/3179 [00:00<?, ?it/s]

In [21]:
pd.crosstab(fp_preds.layer, fp_preds['FP_0.5'], margins=True)

FP_0.5,FP,TP,All
layer,,,
groundwood,1856,976,2832
uprightwood,135,212,347
All,1991,1188,3179


In [22]:
pd.crosstab(tp_truths.layer, tp_truths['TP_0.5'], margins=True)

TP_0.5,FN,TP,All
layer,,,
groundwood,425,976,1401
uprightwood,128,212,340
All,553,1188,1741


From these we can get both precision and recall: $Precision = \frac{tp}{tp+fp}, Recall = \frac{tp}{tp+fn}$

In [23]:
print(f'Precision for fallen deadwood with IoU threshold of 0.5 is {(976/2832):.2f}')
print(f'Recall for fallen deadwood with IoU threshold of 0.5 is {(976/1401):.2f}')

Precision for fallen deadwood with IoU threshold of 0.5 is 0.34
Recall for fallen deadwood with IoU threshold of 0.5 is 0.70


In [24]:
print(f'Precision for standing deadwood with IoU threshold of 0.5 is {(212/347):.2f}')
print(f'Recall for standing deadwood with IoU threshold of 0.5 is {(212/340):.2f}')

Precision for standing deadwood with IoU threshold of 0.5 is 0.61
Recall for standing deadwood with IoU threshold of 0.5 is 0.62


In [26]:
print(f'Overall precision with IoU threshold of 0.5 is {(1188/3179):.2f}')
print(f'Overall recall with IoU threshold of 0.5 is {(1188/1741):.2f}')

Overall precision with IoU threshold of 0.5 is 0.37
Overall recall with IoU threshold of 0.5 is 0.68


## Half patch overlap and edge filtering

For this postprocessing method, mosaics are tiled so that the sliding window moves half tile lenght. For example, when moving row-wise, the first bottom-left coordinates are (0,0), and next ones (256,0), (512,0)... and same  is done column-wise. We discard all predicted polygons whose centroid point is not within the half-overlap area. For instance, for first tile (bottom left (0,0)), the x-coordinate must be between 128 and 384, for second tile (256,0) between 384 and 640, and likewise for y-coordinates. This method discards almost 75% of all predictions in the virtual plots as they are either overlapping or cut in half in the patch borders.

The images used for predictions are buffered so that the whole area is covered, considering the discarding process. 

In [27]:
pred_outpath = Path('../results/hp_overlap_filter/')
if not os.path.exists(pred_outpath):
    shutil.copytree('../results/template_folder/', pred_outpath, symlinks=True)

In [28]:
#| output: false

raw_path = Path('../../data/raw/hiidenportti/virtual_plots/buffered_test/images')
test_rasters = [raw_path/f for f in os.listdir(raw_path) if f.endswith('tif')]

for t in test_rasters:
    outfile_name = pred_outpath/f'raw_preds/{str(t).split("/")[-1][:-4]}.geojson'
    predict_instance_masks(path_to_model_config='../models/hiidenportti/mask_rcnn_R_101_FPN_3x/config.yaml', 
                           path_to_image=str(t),
                           outfile=str(outfile_name),
                           processing_dir='temp',
                           tile_size=512,
                           tile_overlap=256,
                           smooth_preds=False,
                           use_tta=True,
                           coco_set='../../data/processed/hiidenportti/hiidenportti_valid.json',
                           postproc_results=True)

Reading and tiling ../../data/raw/hiidenportti/virtual_plots/buffered_test/images/104_49_Hiidenportti_Chunk5_orto.tif to 512x512 tiles with overlap of 256px


0it [00:00, ?it/s]

Loading model
Starting predictions


  0%|          | 0/276 [00:00<?, ?it/s]

  0%|          | 0/276 [00:00<?, ?it/s]

...

  0%|          | 0/81 [00:00<?, ?it/s]

  0%|          | 0/81 [00:00<?, ?it/s]

698 polygons before edge area removal


  0%|          | 0/55 [00:00<?, ?it/s]

159 polygons before non-max suppression
159 polygons after non-max suppression
Removing intermediate files


Modify as previously.

In [29]:
hp_res_path = pred_outpath
truth_shps = sorted([hp_res_path/'vector_tiles'/f for f in os.listdir(hp_res_path/'vector_tiles')])
hp_raw_shps = sorted([hp_res_path/'raw_preds'/f for f in os.listdir(hp_res_path/'raw_preds')])
rasters = sorted([hp_res_path/'raster_tiles'/f for f in os.listdir(hp_res_path/'raster_tiles')])

In [30]:
for p, t in zip(hp_raw_shps, truth_shps):
    temp_pred = gpd.read_file(p)
    temp_truth = gpd.read_file(t)
    temp_pred['geometry'] = temp_pred.apply(lambda row: fix_multipolys(row.geometry) 
                                            if row.geometry.type == 'MultiPolygon' 
                                            else shapely.geometry.Polygon(row.geometry.exterior), axis=1)
    temp_pred['label'] += 1
    temp_pred = gpd.clip(temp_pred, box(*temp_truth.total_bounds))
    temp_pred = temp_pred[temp_pred.geometry.area > 16*0.04**2]
    temp_pred.to_file(hp_res_path/'predicted_vectors'/p.name)

In [31]:
pred_shps = sorted([hp_res_path/'predicted_vectors'/f for f in os.listdir(hp_res_path/'predicted_vectors')])

Collate all predictions into single dataframes

In [32]:
truths = None
preds = None

for p, t in zip(pred_shps, truth_shps):
    temp_pred = gpd.read_file(p)
    temp_truth = gpd.read_file(t)
    if truths is None:
        truths = temp_truth
        preds = temp_pred
    else:
        truths = pd.concat((truths, temp_truth))
        preds = pd.concat((preds, temp_pred))

In [33]:
preds['layer'] = preds.apply(lambda row: 'groundwood' if row.label == 2 else 'uprightwood', axis=1)

The total amount after cleaning is similar than before.

In [34]:
preds.layer.value_counts()

groundwood     2737
uprightwood     318
Name: layer, dtype: int64

In [35]:
dis_truths = truths.dissolve(by='layer')
dis_preds = preds.dissolve(by='layer')

But IoU, especially for standing deadwood increases.

In [36]:
poly_IoU(dis_truths, dis_preds)

layer
groundwood     0.484690
uprightwood    0.535325
dtype: float64

In [37]:
deadwood_categories = [{'supercategory': 'deadwood', 'id':1, 'name':'uprightwood'},
                 
                       {'supercategory': 'deadwood', 'id':2, 'name':'groundwood'}]

hp_coco_eval = GisCOCOeval(hp_res_path, hp_res_path, 
                            None, None, deadwood_categories)

In [38]:
hp_coco_eval.prepare_data(gt_label_col='layer')

0it [00:00, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [39]:
hp_coco_eval.prepare_eval()

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.03s)
creating index...
index created!


In [40]:
hp_coco_eval.coco_eval.params.maxDets = [1000, 10000]

In [41]:
hp_coco_eval.evaluate()


Evaluating for category uprightwood
Running per image evaluation...
Evaluate annotation type *segm*
DONE (t=0.67s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=10000 ] = 0.327
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=10000 ] = 0.593
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=10000 ] = 0.337
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=10000 ] = 0.167
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=10000 ] = 0.392
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=10000 ] = 1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.406
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.233
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.474
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 1.000

Evaluating for 

Overall AP50 increases by around 0.1 with this kind of post-processing.

In [42]:
tp_truths = truths.copy()
tp_truths.rename(columns={'groundwood':'label'}, inplace=True)
truth_sindex = tp_truths.sindex
fp_preds = preds.copy()
pred_sindex = fp_preds.sindex
tp_truths[tp_cols] = tp_truths.progress_apply(lambda row: is_true_positive(row, fp_preds, pred_sindex), 
                                              axis=1, result_type='expand')
fp_preds[fp_cols] = fp_preds.progress_apply(lambda row: is_false_positive(row, tp_truths, truth_sindex,
                                                                            fp_preds, pred_sindex),
                                            axis=1, result_type='expand')

  0%|          | 0/1741 [00:00<?, ?it/s]

  0%|          | 0/3055 [00:00<?, ?it/s]

In [43]:
pd.crosstab(fp_preds.layer, fp_preds['FP_0.5'], margins=True)

FP_0.5,FP,TP,All
layer,,,
groundwood,1673,1064,2737
uprightwood,91,227,318
All,1764,1291,3055


In [44]:
pd.crosstab(tp_truths.layer, tp_truths['TP_0.5'], margins=True)

TP_0.5,FN,TP,All
layer,,,
groundwood,337,1064,1401
uprightwood,113,227,340
All,450,1291,1741


$Precision = \frac{tp}{tp+fp}, Recall = \frac{tp}{tp+fn}$

In [45]:
print(f'Precision for fallen deadwood with IoU threshold of 0.5 is {(1064/2737):.2f}')
print(f'Recall for fallen deadwood with IoU threshold of 0.5 is {(1064/1401):.2f}')

Precision for fallen deadwood with IoU threshold of 0.5 is 0.39
Recall for fallen deadwood with IoU threshold of 0.5 is 0.76


In [46]:
print(f'Precision for standing deadwood with IoU threshold of 0.5 is {(227/318):.2f}')
print(f'Recall for standing deadwood with IoU threshold of 0.5 is {(227/318):.2f}')

Precision for standing deadwood with IoU threshold of 0.5 is 0.71
Recall for standing deadwood with IoU threshold of 0.5 is 0.71


In [47]:
print(f'Overall precision with IoU threshold of 0.5 is {(1291/3055):.2f}')
print(f'Overall recall with IoU threshold of 0.5 is {(1291/1741):.2f}')

Overall precision with IoU threshold of 0.5 is 0.42
Overall recall with IoU threshold of 0.5 is 0.74


## Overlap, edge filtering and mask merging

Mask merging is built on previous predictions. In this step, for each polygon we check whether the ratio between intersection with any other polygon of the same class and the area of the polygon is more than 0.2. If yes, the polygon is merged to the other polygon with which it had intersection-over-area ratio.

In [48]:
merge_outpath = Path('../results/hp_merge/')
if not os.path.exists(merge_outpath):
    shutil.copytree('../results/template_folder/', merge_outpath, symlinks=True)

Two iterations of merging is usually enough.

In [49]:
#| output: false

for r in pred_shps:
    gdf_temp = gpd.read_file(r)
    standing = gdf_temp[gdf_temp.label==1].copy()
    fallen = gdf_temp[gdf_temp.label==2].copy()
    standing = merge_polys(standing, 0.2)
    fallen = merge_polys(fallen, 0.2)
    standing = merge_polys(standing, 0.2)
    fallen = merge_polys(fallen, 0.2)
    gdf_merged = pd.concat((standing, fallen))
    gdf_merged.to_file(merge_outpath/'predicted_vectors'/r.name, driver='GeoJSON')
    gdf_merged = None
    gdf_temp = None

175it [00:00, 432.85it/s]
1124it [00:05, 209.10it/s]
163it [00:00, 496.21it/s]
848it [00:02, 299.31it/s]
16it [00:00, 502.27it/s]
131it [00:00, 351.82it/s]
15it [00:00, 614.55it/s]
98it [00:00, 544.72it/s]
20it [00:00, 495.00it/s]
389it [00:01, 288.98it/s]
19it [00:00, 591.11it/s]
304it [00:00, 374.01it/s]
103it [00:00, 529.07it/s]
945it [00:04, 233.92it/s]
101it [00:00, 569.06it/s]
750it [00:02, 298.61it/s]
4it [00:00, 583.82it/s]
148it [00:00, 377.84it/s]
4it [00:00, 606.35it/s]
122it [00:00, 544.98it/s]


In [50]:
merged_coco_eval = GisCOCOeval(merge_outpath, merge_outpath, None, None, deadwood_categories)
merged_coco_eval.prepare_data(gt_label_col='layer')
merged_coco_eval.prepare_eval()
merged_coco_eval.evaluate()

0it [00:00, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.02s)
creating index...
index created!

Evaluating for category uprightwood
Running per image evaluation...
Evaluate annotation type *segm*
DONE (t=0.63s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.324
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.597
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.326
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.164
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.390
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.347
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.189
 Average Recall     (A

This usually worsens the results a bit, but the produced results are better suited for deriving forest charasteristics, as the number of overlapping detected instances decrease significantly.

In [51]:
preds.reset_index(drop=True, inplace=True)
standing = merge_polys(preds[preds.label == 1].copy(), 0.2)
fallen = merge_polys(preds[preds.label == 2].copy(), 0.2)
standing = merge_polys(standing, 0.2)
fallen = merge_polys(fallen, 0.2)
preds_merged = pd.concat((standing, fallen))

318it [00:00, 405.32it/s]
2737it [00:21, 129.54it/s]
302it [00:00, 449.91it/s]
2122it [00:12, 172.92it/s]


In [52]:
preds_merged['layer'] = preds_merged.apply(lambda row: 'groundwood' if row.label == 2 else 'uprightwood', axis=1)
preds_merged.layer.value_counts()

groundwood     2042
uprightwood     300
Name: layer, dtype: int64

In [53]:
tp_truths = truths.copy()
tp_truths.rename(columns={'groundwood':'label'}, inplace=True)
truth_sindex = tp_truths.sindex
fp_preds_merged = preds_merged.copy()
pred_sindex = fp_preds_merged.sindex
tp_truths[tp_cols] = tp_truths.progress_apply(lambda row: is_true_positive(row, fp_preds_merged, pred_sindex), 
                                              axis=1, result_type='expand')
fp_preds_merged[fp_cols] = fp_preds_merged.progress_apply(lambda row: is_false_positive(row, tp_truths, truth_sindex,
                                                                            fp_preds_merged, pred_sindex),
                                            axis=1, result_type='expand')

  0%|          | 0/1741 [00:00<?, ?it/s]

  0%|          | 0/2342 [00:00<?, ?it/s]

In [54]:
pd.crosstab(fp_preds_merged.layer, fp_preds_merged['FP_0.5'], margins=True)

FP_0.5,FP,TP,All
layer,,,
groundwood,1079,963,2042
uprightwood,74,226,300
All,1153,1189,2342


In [55]:
pd.crosstab(tp_truths.layer, tp_truths['TP_0.5'], margins=True)

TP_0.5,FN,TP,All
layer,,,
groundwood,438,963,1401
uprightwood,114,226,340
All,552,1189,1741


$Precision = \frac{tp}{tp+fp}, Recall = \frac{tp}{tp+fn}$

In [56]:
print(f'Precision for fallen deadwood with IoU threshold of 0.5 is {(963/2042):.2f}')
print(f'Recall for fallen deadwood with IoU threshold of 0.5 is {(963/1401):.2f}')

Precision for fallen deadwood with IoU threshold of 0.5 is 0.47
Recall for fallen deadwood with IoU threshold of 0.5 is 0.69


In [57]:
print(f'Precision for standing deadwood with IoU threshold of 0.5 is {(226/300):.2f}')
print(f'Recall for standing deadwood with IoU threshold of 0.5 is {(226/340):.2f}')

Precision for standing deadwood with IoU threshold of 0.5 is 0.75
Recall for standing deadwood with IoU threshold of 0.5 is 0.66


In [58]:
print(f'Overall precision with IoU threshold of 0.5 is {(1189/2342):.2f}')
print(f'Overall recall with IoU threshold of 0.5 is {(1189/1741):.2f}')

Overall precision with IoU threshold of 0.5 is 0.51
Overall recall with IoU threshold of 0.5 is 0.68


As the postprocessing merges data instead of dropping less certain predictions, total area and IoU remain the same as in previous step.

In [59]:
preds_merged.to_file('../results/hiidenportti/merged_all_new.geojson')

# Full Evo dataset

Running predictions for Evo dataset takes so much time that it has been done separately. 

## No overlap, no post-processing

In [60]:
spk_raw_res_path = Path('../results/spk_raw/')
truth_shps = sorted([spk_raw_res_path/'vector_tiles'/f for f in os.listdir(spk_raw_res_path/'vector_tiles')])
spk_raw_shps = sorted([spk_raw_res_path/'raw_preds'/f for f in os.listdir(spk_raw_res_path/'raw_preds')])
rasters = sorted([spk_raw_res_path/'raster_tiles'/f for f in os.listdir(spk_raw_res_path/'raster_tiles')])

In [61]:
for p, t in zip(spk_raw_shps, truth_shps):
    temp_pred = gpd.read_file(p)
    temp_truth = gpd.read_file(t)
    temp_pred['geometry'] = temp_pred.apply(lambda row: fix_multipolys(row.geometry) 
                                            if row.geometry.type == 'MultiPolygon' 
                                            else shapely.geometry.Polygon(row.geometry.exterior), axis=1)
    temp_pred['label'] += 1
    temp_pred = gpd.clip(temp_pred, box(*temp_truth.total_bounds))
    temp_pred = temp_pred[temp_pred.geometry.area > 16*0.0485**2]
    temp_pred.to_file(spk_raw_res_path/'predicted_vectors'/p.name)

In [62]:
pred_shps = sorted([spk_raw_res_path/'predicted_vectors'/f for f in os.listdir(spk_raw_res_path/'predicted_vectors')])

In [63]:
truths = None
preds = None

for p, t in zip(pred_shps, truth_shps):
    temp_pred = gpd.read_file(p)
    temp_truth = gpd.read_file(t)
    if truths is None:
        truths = temp_truth
        preds = temp_pred
    else:
        truths = pd.concat((truths, temp_truth))
        preds = pd.concat((preds, temp_pred))

In [64]:
preds['layer'] = preds.apply(lambda row: 'groundwood' if row.label == 2 else 'uprightwood', axis=1)

In [65]:
preds.shape, truths.shape

((6079, 4), (5334, 4))

In [66]:
preds.layer.value_counts()

groundwood     4761
uprightwood    1318
Name: layer, dtype: int64

In [67]:
truths.rename(columns={'label':'layer'}, inplace=True)

In [68]:
dis_truths = truths.dissolve(by='layer')
dis_preds = preds.dissolve(by='layer')

IoU for standing deadwood is good already for this preprocessing level.

In [69]:
poly_IoU(dis_truths, dis_preds)

layer
groundwood     0.447793
uprightwood    0.588185
dtype: float64

In [70]:
deadwood_categories = [{'supercategory': 'deadwood', 'id':1, 'name':'uprightwood'},
                 
                       {'supercategory': 'deadwood', 'id':2, 'name':'groundwood'}]

spk_raw_coco_eval = GisCOCOeval(spk_raw_res_path, spk_raw_res_path, 
                           None, None, deadwood_categories)
spk_raw_coco_eval.prepare_data(gt_label_col='label')
spk_raw_coco_eval.prepare_eval()
spk_raw_coco_eval.coco_eval.params.maxDets = [1000, 10000]
spk_raw_coco_eval.evaluate()

0it [00:00, ?it/s]

  0%|          | 0/71 [00:00<?, ?it/s]

loading annotations into memory...
Done (t=0.19s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.05s)
creating index...
index created!

Evaluating for category uprightwood
Running per image evaluation...
Evaluate annotation type *segm*
DONE (t=1.10s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=10000 ] = 0.233
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=10000 ] = 0.478
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=10000 ] = 0.205
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=10000 ] = 0.087
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=10000 ] = 0.311
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=10000 ] = 0.420
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.331
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.182
 Average Recal

In [71]:
#| output: false

tp_truths = truths.copy()
tp_truths['label'] = tp_truths.layer.apply(lambda row: 1 if row=='uprightwood' else 2)
truth_sindex = tp_truths.sindex
fp_preds = preds.copy()
pred_sindex = fp_preds.sindex
tp_truths[tp_cols] = tp_truths.progress_apply(lambda row: is_true_positive(row, fp_preds, pred_sindex), 
                                              axis=1, result_type='expand')
fp_preds[fp_cols] = fp_preds.progress_apply(lambda row: is_false_positive(row, tp_truths, truth_sindex,
                                                                            fp_preds, pred_sindex),
                                            axis=1, result_type='expand')

  0%|          | 0/5334 [00:00<?, ?it/s]

  0%|          | 0/6079 [00:00<?, ?it/s]

In [72]:
pd.crosstab(fp_preds.layer, fp_preds['FP_0.5'], margins=True)

FP_0.5,FP,TP,All
layer,,,
groundwood,2873,1888,4761
uprightwood,501,817,1318
All,3374,2705,6079


In [73]:
pd.crosstab(tp_truths.layer, tp_truths['TP_0.5'], margins=True)

TP_0.5,FN,TP,All
layer,,,
groundwood,2027,1888,3915
uprightwood,602,817,1419
All,2629,2705,5334


$Precision = \frac{tp}{tp+fp}, Recall = \frac{tp}{tp+fn}$

In [74]:
print(f'Precision for fallen deadwood with IoU threshold of 0.5 is {(1888/4761):.2f}')
print(f'Recall for fallen deadwood with IoU threshold of 0.5 is {(1888/3915):.2f}')

Precision for fallen deadwood with IoU threshold of 0.5 is 0.40
Recall for fallen deadwood with IoU threshold of 0.5 is 0.48


In [75]:
print(f'Precision for standing deadwood with IoU threshold of 0.5 is {(817/1318):.2f}')
print(f'Recall for standing deadwood with IoU threshold of 0.5 is {(817/1419):.2f}')

Precision for standing deadwood with IoU threshold of 0.5 is 0.62
Recall for standing deadwood with IoU threshold of 0.5 is 0.58


In [76]:
print(f'Overall precision with IoU threshold of 0.5 is {(2705/6079):.2f}')
print(f'Overall recall with IoU threshold of 0.5 is {(2705/5334):.2f}')

Overall precision with IoU threshold of 0.5 is 0.44
Overall recall with IoU threshold of 0.5 is 0.51


## Half patch overlap and edge filtering

In [77]:
spk_res_path = Path('../results/spk_buffered/')
truth_shps = sorted([spk_res_path/'vector_tiles'/f for f in os.listdir(spk_res_path/'vector_tiles')])
spk_buf_raw_shps = sorted([spk_res_path/'raw_preds'/f for f in os.listdir(spk_res_path/'raw_preds')])
rasters = sorted([spk_res_path/'raster_tiles'/f for f in os.listdir(spk_res_path/'raster_tiles')])

In [78]:
for p, t in zip(spk_buf_raw_shps, truth_shps):
    temp_pred = gpd.read_file(p)
    temp_truth = gpd.read_file(t)
    temp_pred = gpd.clip(temp_pred, box(*temp_truth.total_bounds))
    temp_pred['geometry'] = temp_pred.apply(lambda row: fix_multipolys(row.geometry) 
                                            if row.geometry.type == 'MultiPolygon' 
                                            else shapely.geometry.Polygon(row.geometry.exterior), axis=1)
    temp_pred['label'] += 1
    temp_pred = temp_pred[temp_pred.geometry.area > 16*0.0485**2]
    temp_pred.to_file(spk_res_path/'predicted_vectors'/p.name)

In [79]:
pred_shps = sorted([spk_res_path/'predicted_vectors'/f for f in os.listdir(spk_res_path/'predicted_vectors')])

In [80]:
truths = None
preds = None

for p, t in zip(pred_shps, truth_shps):
    temp_pred = gpd.read_file(p)
    temp_truth = gpd.read_file(t)
    if truths is None:
        truths = temp_truth
        preds = temp_pred
    else:
        truths = pd.concat((truths, temp_truth))
        preds = pd.concat((preds, temp_pred))

In [81]:
preds['layer'] = preds.apply(lambda row: 'groundwood' if row.label == 2 else 'uprightwood', axis=1)

In [82]:
preds.shape, truths.shape

((6194, 4), (5334, 4))

In [83]:
preds.layer.value_counts()

groundwood     4908
uprightwood    1286
Name: layer, dtype: int64

In [84]:
truths.rename(columns={'label':'layer'}, inplace=True)

In [85]:
dis_truths = truths.dissolve(by='layer')
dis_preds = preds.dissolve(by='layer')

In [86]:
poly_IoU(dis_truths, dis_preds)

layer
groundwood     0.458600
uprightwood    0.607651
dtype: float64

In [87]:
deadwood_categories = [{'supercategory': 'deadwood', 'id':1, 'name':'uprightwood'},
                 
                       {'supercategory': 'deadwood', 'id':2, 'name':'groundwood'}]

spk_coco_eval = GisCOCOeval(spk_res_path, spk_res_path, 
                           None, None, deadwood_categories)
spk_coco_eval.prepare_data(gt_label_col='label')
spk_coco_eval.prepare_eval()
spk_coco_eval.coco_eval.params.maxDets = [1000, 10000]
spk_coco_eval.evaluate()

0it [00:00, ?it/s]

  0%|          | 0/71 [00:00<?, ?it/s]

loading annotations into memory...
Done (t=0.19s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.05s)
creating index...
index created!

Evaluating for category uprightwood
Running per image evaluation...
Evaluate annotation type *segm*
DONE (t=1.06s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=10000 ] = 0.300
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=10000 ] = 0.551
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=10000 ] = 0.304
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=10000 ] = 0.125
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=10000 ] = 0.404
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=10000 ] = 0.368
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.384
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.217
 Average Recal

The overall AP50 score is close to the patch-level results here.

In [88]:
tp_truths = truths.copy()
tp_truths['label'] = tp_truths.layer.apply(lambda row: 1 if row=='uprightwood' else 2)
truth_sindex = tp_truths.sindex
fp_preds = preds.copy()
pred_sindex = fp_preds.sindex
tp_truths[tp_cols] = tp_truths.progress_apply(lambda row: is_true_positive(row, fp_preds, pred_sindex), 
                                              axis=1, result_type='expand')
fp_preds[fp_cols] = fp_preds.progress_apply(lambda row: is_false_positive(row, tp_truths, truth_sindex,
                                                                            fp_preds, pred_sindex),
                                            axis=1, result_type='expand')

  0%|          | 0/5334 [00:00<?, ?it/s]

  0%|          | 0/6194 [00:00<?, ?it/s]

In [89]:
pd.crosstab(fp_preds.layer, fp_preds['FP_0.5'], margins=True)

FP_0.5,FP,TP,All
layer,,,
groundwood,2818,2090,4908
uprightwood,396,890,1286
All,3214,2980,6194


In [90]:
pd.crosstab(tp_truths.layer, tp_truths['TP_0.5'], margins=True)

TP_0.5,FN,TP,All
layer,,,
groundwood,1824,2091,3915
uprightwood,529,890,1419
All,2353,2981,5334


$Precision = \frac{tp}{tp+fp}, Recall = \frac{tp}{tp+fn}$

In [91]:
print(f'Precision for fallen deadwood with IoU threshold of 0.5 is {(2090/4098):.2f}')
print(f'Recall for fallen deadwood with IoU threshold of 0.5 is {(2091/3915):.2f}')

Precision for fallen deadwood with IoU threshold of 0.5 is 0.51
Recall for fallen deadwood with IoU threshold of 0.5 is 0.53


In [92]:
print(f'Precision for standing deadwood with IoU threshold of 0.5 is {(890/1286):.2f}')
print(f'Recall for standing deadwood with IoU threshold of 0.5 is {(890/1419):.2f}')

Precision for standing deadwood with IoU threshold of 0.5 is 0.69
Recall for standing deadwood with IoU threshold of 0.5 is 0.63


In [93]:
print(f'Overall precision with IoU threshold of 0.5 is {(2980/6194):.2f}')
print(f'Overall recall with IoU threshold of 0.5 is {(2981/5334):.2f}')

Overall precision with IoU threshold of 0.5 is 0.48
Overall recall with IoU threshold of 0.5 is 0.56


## Overlap, edge filtering and mask merging

In [94]:
merge_outpath = Path('../results/spk_merge/')
if not os.path.exists(merge_outpath):
    shutil.copytree('../results/spk_template/', merge_outpath, symlinks=True)

Two iterations of merging is usually enough.

In [ ]:
for r in pred_shps:
    gdf_temp = gpd.read_file(r)
    standing = gdf_temp[gdf_temp.label==1].copy()
    fallen = gdf_temp[gdf_temp.label==2].copy()
    if len(standing) > 0:
        standing = merge_polys(standing, 0.2)
        standing = merge_polys(standing, 0.2)
    if len(fallen) > 0:
        fallen = merge_polys(fallen, 0.2)
        fallen = merge_polys(fallen, 0.2)
    gdf_merged = pd.concat((standing, fallen))
    gdf_merged.to_file(merge_outpath/'predicted_vectors'/r.name, driver='GeoJSON')
    gdf_merged = None
    gdf_temp = None

In [96]:
merged_coco_eval = GisCOCOeval(merge_outpath, merge_outpath, None, None, deadwood_categories)
merged_coco_eval.prepare_data(gt_label_col='label')
merged_coco_eval.prepare_eval()
merged_coco_eval.evaluate()

0it [00:00, ?it/s]

  0%|          | 0/71 [00:00<?, ?it/s]

loading annotations into memory...
Done (t=0.19s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.04s)
creating index...
index created!

Evaluating for category uprightwood
Running per image evaluation...
Evaluate annotation type *segm*
DONE (t=0.95s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.286
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.530
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.287
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.123
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.384
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.345
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.357
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.209
 Average Recall     (A

In [97]:
preds.reset_index(drop=True, inplace=True)
standing = merge_polys(preds[preds.label == 1].copy(), 0.2)
fallen = merge_polys(preds[preds.label == 2].copy(), 0.2)
standing = merge_polys(standing, 0.2)
fallen = merge_polys(fallen, 0.2)
preds_merged = pd.concat((standing, fallen))

1286it [00:05, 217.24it/s]
4908it [00:57, 85.47it/s]
1176it [00:04, 241.19it/s]
4020it [00:37, 108.54it/s]


In [98]:
preds_merged['layer'] = preds_merged.apply(lambda row: 'groundwood' if row.label == 2 else 'uprightwood', axis=1)
preds_merged.layer.value_counts()

groundwood     3923
uprightwood    1159
Name: layer, dtype: int64

In [99]:
tp_truths = truths.copy()
tp_truths['label'] = tp_truths.layer.apply(lambda row: 1 if row=='uprightwood' else 2)
truth_sindex = tp_truths.sindex
fp_preds_merged = preds_merged.copy()
pred_sindex = fp_preds_merged.sindex
tp_truths[tp_cols] = tp_truths.progress_apply(lambda row: is_true_positive(row, fp_preds_merged, pred_sindex), 
                                              axis=1, result_type='expand')
fp_preds_merged[fp_cols] = fp_preds_merged.progress_apply(lambda row: is_false_positive(row, tp_truths, truth_sindex,
                                                                            fp_preds_merged, pred_sindex),
                                            axis=1, result_type='expand')

  0%|          | 0/5334 [00:00<?, ?it/s]

  0%|          | 0/5082 [00:00<?, ?it/s]

In [100]:
pd.crosstab(fp_preds_merged.layer, fp_preds_merged['FP_0.5'], margins=True)

FP_0.5,FP,TP,All
layer,,,
groundwood,1900,2023,3923
uprightwood,321,838,1159
All,2221,2861,5082


In [101]:
pd.crosstab(tp_truths.layer, tp_truths['TP_0.5'], margins=True)

TP_0.5,FN,TP,All
layer,,,
groundwood,1891,2024,3915
uprightwood,581,838,1419
All,2472,2862,5334


$Precision = \frac{tp}{tp+fp}, Recall = \frac{tp}{tp+fn}$

In [102]:
print(f'Precision for fallen deadwood with IoU threshold of 0.5 is {(2023/3923):.2f}')
print(f'Recall for fallen deadwood with IoU threshold of 0.5 is {(2024/3915):.2f}')

Precision for fallen deadwood with IoU threshold of 0.5 is 0.52
Recall for fallen deadwood with IoU threshold of 0.5 is 0.52


In [103]:
print(f'Precision for standing deadwood with IoU threshold of 0.5 is {(838/1159):.2f}')
print(f'Recall for standing deadwood with IoU threshold of 0.5 is {(838/1419):.2f}')

Precision for standing deadwood with IoU threshold of 0.5 is 0.72
Recall for standing deadwood with IoU threshold of 0.5 is 0.59


In [104]:
print(f'Overall precision with IoU threshold of 0.5 is {(2861/5082):.2f}')
print(f'Overall recall with IoU threshold of 0.5 is {(2862/5334):.2f}')

Overall precision with IoU threshold of 0.5 is 0.56
Overall recall with IoU threshold of 0.5 is 0.54


In [105]:
#| echo: false
preds_merged.to_file('../results/sudenpesankangas/spk_merged_new.geojson')